<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/PFI_RRI_RQ_RI_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Exact UFI, PFI, RRI, RQ, and RI for a 2x2 table calculator
# Note that this uses Pearson chi-square, two-sided, no Yates.
# Defaults alpha to 0.05.
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
#
# Heston, T. F. (2025). Fragility Metrics Toolkit (Version 2.1.0) [Software].
# Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0
#

import numpy as np
from scipy.stats import chi2_contingency, chi2

ALPHA_DEFAULT = 0.05  # hard default

# ---------- Core utilities ----------

def chi2_p_pearson(a, b, c, d):
    """
    Two-sided Pearson chi-square (no Yates). Returns (chi2_stat, p_value).
    """
    tbl = np.array([[a, b], [c, d]], dtype=float)
    if np.any(tbl < 0) or not np.isfinite(tbl.sum()) or tbl.sum() == 0:
        return (np.nan, np.nan)
    chi2_stat, p, dof, exp = chi2_contingency(tbl, correction=False)
    return float(chi2_stat), float(p)

def p_value_pearson(a, b, c, d):
    _, p = chi2_p_pearson(a, b, c, d)
    return p

def p_along_x(a, b, c, d, x):
    """
    Fixed-margins path: (a, b, c, d) -> (a+x, b-x, c-x, d+x)
    """
    aa, bb, cc, dd = a + x, b - x, c - x, d + x
    if min(aa, bb, cc, dd) < 0:
        return np.nan
    return p_value_pearson(aa, bb, cc, dd)

def compute_ufi_pfi(a, b, c, d, alpha=ALPHA_DEFAULT, tol=1e-10, max_iter=200):
    """
    Exact UFI then PFI for a 2x2 table along the fixed-margins path.
    UFI = minimal |x| that flips significance (two-sided Pearson chi-square, no Yates).
    PFI = |4x| / n.
    Also returns RRI and RQ.
    """
    a, b, c, d = map(float, (a, b, c, d))
    n = a + b + c + d
    if n <= 0:
        raise ValueError("Total n must be > 0.")
    if min(a, b, c, d) < 0:
        raise ValueError("All cells must be >= 0.")

    # RRI and RQ (p-value independent)
    rri = abs(a*d - b*c) / n
    rq  = (4.0 * rri) / n

    # Feasible x to keep all cells >= 0
    x_min = -min(a, d)      # left bound (<= 0)
    x_max =  min(b, c)      # right bound (>= 0)

    # Original chi2, p and state
    chi2_0, p0 = chi2_p_pearson(a, b, c, d)
    if not np.isfinite(p0):
        raise ValueError("Invalid table for chi-square.")
    sig0 = (p0 <= alpha)

    # Independence location along path
    r1, r2 = a + b, c + d
    c1, c2 = a + c, b + d
    a_exp = r1 * c1 / n
    x_ind = a_exp - a  # displacement to independence

    # Target function
    def f(x):
        return p_along_x(a, b, c, d, x) - alpha

    # Choose search interval toward flip
    if sig0:  # raise p toward independence
        lo, hi = (0.0, x_ind) if x_ind >= 0 else (x_ind, 0.0)
    else:     # lower p away from independence
        if x_ind >= 0:
            lo, hi = (0.0, x_min)  # x_min <= 0
        else:
            lo, hi = (0.0, x_max)  # x_max >= 0

    # Order, clip to feasibility
    if lo > hi:
        lo, hi = hi, lo
    lo = max(lo, x_min)
    hi = min(hi, x_max)
    if lo == hi:
        return {
            "inputs": {"a": a, "b": b, "c": c, "d": d},
            "alpha": alpha,
            "p_original": p0,
            "x_signed": None,
            "UFI": None,
            "PFI_decimal": None,
            "PFI_percent": None,
            "p_at_flip": None,
            "RRI": rri,
            "RQ": rq,
            "RI": None,
            "RI_mode": None
        }

    # Bracket sign change; grid if needed
    flo, fhi = f(lo), f(hi)
    if not (np.isfinite(flo) and np.isfinite(fhi)) or flo * fhi > 0:
        GRID = 400
        xs = np.linspace(lo, hi, GRID + 1)
        fs = np.array([f(x) for x in xs])
        found = False
        for i in range(GRID):
            if np.isfinite(fs[i]) and np.isfinite(fs[i+1]) and fs[i] * fs[i+1] <= 0:
                lo, hi = xs[i], xs[i+1]
                flo, fhi = fs[i], fs[i+1]
                found = True
                break
        if not found:
            return {
                "inputs": {"a": a, "b": b, "c": c, "d": d},
                "alpha": alpha,
                "p_original": p0,
                "x_signed": None,
                "UFI": None,
                "PFI_decimal": None,
                "PFI_percent": None,
                "p_at_flip": None,
                "RRI": rri,
                "RQ": rq,
                "RI": None,
                "RI_mode": None
            }

    # Bisection
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        fmid = f(mid)
        if not np.isfinite(fmid):
            mid = np.nextafter(mid, hi)
            fmid = f(mid)
            if not np.isfinite(fmid):
                break
        if abs(hi - lo) < tol:
            x_star = mid
            break
        if flo == 0:
            x_star = lo
            break
        if fhi == 0:
            x_star = hi
            break
        if flo * fmid <= 0:
            hi, fhi = mid, fmid
        else:
            lo, flo = mid, fmid
    else:
        x_star = 0.5 * (lo + hi)

    p_flip = p_along_x(a, b, c, d, x_star)
    UFI = abs(x_star)
    PFI_decimal = (4.0 * UFI) / n
    PFI_percent = 100.0 * PFI_decimal

    # ---------- Robustness Index (RI) ----------
    # For fixed proportions, Pearson chi-square scales ~ linearly with sample size.
    # Let chi2_crit be the 1-alpha quantile for df=1.
    chi2_crit = chi2.ppf(1.0 - alpha, df=1)
    if np.isfinite(chi2_0) and chi2_0 > 0:
        if p0 > alpha:
            # non-significant: multiply all cells by k until chi2(k) >= chi2_crit
            RI = chi2_crit / chi2_0
            RI_mode = "multiply cells by k"
        else:
            # significant: divide all cells by k until chi2(k) < chi2_crit
            RI = chi2_0 / chi2_crit
            RI_mode = "divide cells by k"
        if RI <= 1:  # numerical guard; enforce k > 1
            RI = 1.0 + 0.0
    else:
        RI = None
        RI_mode = None

    return {
        "inputs": {"a": a, "b": b, "c": c, "d": d},
        "alpha": alpha,
        "p_original": p0,
        "x_signed": x_star,
        "UFI": UFI,
        "PFI_decimal": PFI_decimal,
        "PFI_percent": PFI_percent,
        "p_at_flip": p_flip,
        "RRI": rri,
        "RQ": rq,
        "RI": RI,
        "RI_mode": RI_mode
    }

# ---------- Single-run interactive ----------

def _fmt(x, fmt):
    return "None" if x is None else (fmt % x)

def run_once():
    print("Enter 2x2 counts (nonnegative). Press Enter to accept default in brackets.")
    def read_float(prompt, default=None):
        s = input(prompt).strip()
        if s == "" and default is not None:
            return float(default)
        return float(s)

    a = read_float("a [30]: ", 30)
    b = read_float("b [55]: ", 55)
    c = read_float("c [17]: ", 17)
    d = read_float("d [20]: ", 20)

    alpha = ALPHA_DEFAULT

    res = compute_ufi_pfi(a, b, c, d, alpha=alpha)

    print("\n--- Results ---")
    print(f"Inputs: a={res['inputs']['a']:.6g}, b={res['inputs']['b']:.6g}, c={res['inputs']['c']:.6g}, d={res['inputs']['d']:.6g}")
    print(f"Alpha: {res['alpha']}")
    print("Original p-value:", _fmt(res['p_original'], "%.10f"))
    print("Signed UFI (x):",  _fmt(res['x_signed'],   "%.10f"))
    print("UFI (|x|):",       _fmt(res['UFI'],        "%.10f"))
    print("PFI (decimal):",   _fmt(res['PFI_decimal'], "%.10f"))
    print("PFI (%):",         _fmt(res['PFI_percent'], "%.6f"))
    print("RRI:",             _fmt(res['RRI'],        "%.10f"))
    print("RQ:",              _fmt(res['RQ'],         "%.10f"))
    print("p-value at flip:", _fmt(res['p_at_flip'],  "%.10f"))
    print("RI (k):",          _fmt(res['RI'],         "%.10f"))
    print("RI mode:",         res['RI_mode'] if res['RI_mode'] is not None else "None")
    print("----------------")

if __name__ == "__main__":
    run_once()


Enter 2x2 counts (nonnegative). Press Enter to accept default in brackets.
a [30]: 3
b [55]: 3
c [17]: 11
d [20]: 10

--- Results ---
Inputs: a=3, b=3, c=11, d=10
Alpha: 0.05
Original p-value: 0.9180109660
Signed UFI (x): -2.0044394598
UFI (|x|): 2.0044394598
PFI (decimal): 0.2969539940
PFI (%): 29.695399
RRI: 0.1111111111
RQ: 0.0164609053
p-value at flip: 0.0500000000
RI (k): 362.5198916714
RI mode: multiply cells by k
----------------
